In [34]:
# Import the data

import pandas as pd

df = pd.read_csv('../data/priest_popular_archetype_decks.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35867 entries, 0 to 35866
Columns: 1275 entries, deck_archetype to Murloc Holmes
dtypes: int64(1274), object(1)
memory usage: 348.9+ MB


In [35]:
df.head()

,deck_archetype,Circle of Healing,Flash Heal,Northshire Cleric,Power Word: Shield,Embrace the Shadow,Mind Blast,Shadow Word: Death,Shadow Word: Pain,Auchenai Soulpriest,...,Coilfang Constrictor,Snapdragon,Neptulon the Tidehunter,Ozumat,Prince Renathal,Ethereal Augmerchant,Replicat-o-tron,Cathedral of Atonement,Dispossessed Soul,Murloc Holmes
0,Control Priest,2,2,2,2,2,2,1,1,2,...,0,0,0,0,0,0,0,0,0,0
1,Dragon Priest,0,0,2,2,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Control Priest,2,0,2,2,0,0,2,0,2,...,0,0,0,0,0,0,0,0,0,0
3,Dragon Priest,0,0,2,2,0,0,2,1,0,...,0,0,0,0,0,0,0,0,0,0
4,C'Thun Priest,0,0,2,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
# Prepare Input (X) and Output (y)
X = df.drop(columns=['deck_archetype'])
y = df['deck_archetype']

In [37]:
# Encode Labels (deck_archetype)

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

In [38]:
# Split the data for training

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.25, random_state=42)

In [39]:
# Build Neural Network Model
# Note: have to use python 3.12.8 because tensorflow does not yet handle versions above this one.

from tensorflow import keras

model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.4), # Prevent overfitting
    keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.35),
    keras.layers.Dense(32, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dropout(0.30),
    keras.layers.Dense(len(encoder.classes_), activation='softmax') # Multi-class output
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0005),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

c:\Users\willy\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [40]:
# Model training

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=25,
    restore_best_weights=True
)
model.fit(X_train, y_train, epochs=200, validation_split=0.25, batch_size=16, callbacks=[early_stopping])

Epoch 1/200
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.3352 - loss: 4.0608 - val_accuracy: 0.6980 - val_loss: 1.9824
Epoch 2/200
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.6017 - loss: 2.1193 - val_accuracy: 0.7215 - val_loss: 1.4078
Epoch 3/200
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.6443 - loss: 1.6065 - val_accuracy: 0.7228 - val_loss: 1.2122
Epoch 4/200
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.6694 - loss: 1.3984 - val_accuracy: 0.7312 - val_loss: 1.1112
Epoch 5/200
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.6756 - loss: 1.3087 - val_accuracy: 0.7332 - val_loss: 1.0665
Epoch 6/200
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.6770 - loss: 1.2736 - val_accuracy: 0.7320 - val_loss: 1.0414
Epoch 7/200
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.6822 - loss: 1.2449 - val_accuracy: 0.7312 - val_loss: 1.0432
Epoch 8/200
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.6751 - loss: 1

In [41]:
# Model evaluation

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.2f}")

281/281 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7250 - loss: 1.0224
Test Accuracy: 0.72


In [42]:
# Export the model and the encoder

import joblib

model.save('../models/fnn_dense_layers_model.keras')
joblib.dump(encoder, '../models/fnn_dense_layers_encoder.pkl')

['../models/fnn_dense_layers_encoder.pkl']